# Parallel jieba

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
import jieba,re
import pandas as pd

def imap_unordered_bar(func, args, n_processes=2):  # multi-processing function
    
    p = Pool(n_processes)
    res_list = {}
    with tqdm(total=1, dynamic_ncols=True) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args)), disable=True):
            pbar.update()
            res_list.update(res)
    pbar.close()
    p.close()
    p.join()

    return res_list


def stopwordslist(filepath):  # read stopwords
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords


def jieba_seg_parallel(data, stopwords, cid):  # jieba
    import jieba
    row = []
    dic = {}
   
    seg_list = jieba.cut(data)

    for w in seg_list:
        if w not in stopwords:
            row.append(w)
    dic[cid] = row

    return dic

def all_run(content, stopword):
    
    global jieba
    def jieba(self):
        result = jieba_seg_parallel(self[0], stopword, self[1])
        return result
    
    done_x = imap_unordered_bar(jieba, content, 1)
    
    return done_x

def remove_punctuation(getdata):  # regular

    rule = re.compile("[^a-zA-Z0-9\u4e00-\u9fa5]")
    line = rule.sub('', getdata)

    return line


def seg(df,to_jieba_data, stopwords,seg_type):  # add new colum jieba
    word_dict = all_run(to_jieba_data, stopwords)  # to_jieba_data
    keys = list(word_dict.keys())
    values = list(word_dict.values())
    jieba_df = pd.DataFrame(keys, columns=['id'])
    jieba_df[seg_type] = values
    get_df = pd.merge(df, jieba_df, on=['id'])
    return get_df

if __name__ == "__main__":
    stopwords = stopwordslist('read your stop words')
   
    df_seg = seg(new_df,new_df[['your content','your content id']].values, stopwords,'seg')

# Parallel DTW

In [ ]:
from multiprocessing import Pool
from tqdm import tqdm
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

def imap_unordered_bar(func, args, n_processes = 2): # multi-processing function
    p = Pool(n_processes)
    res_list = []
    with tqdm(total = 1, dynamic_ncols=True) as pbar:
        for i, res in tqdm(enumerate(p.imap_unordered(func, args)), disable=True):
            pbar.update()
            res_list.append(res)
    pbar.close()
    p.close()
    p.join()
    return res_list

def dtw_parallel(self,pair):

    distance, path = fastdtw(self[0],self[1] , dist=euclidean)
    return [distance, pair[0],pair[1]]

def all_run_dtw(pairdict, pair):

    global cal_fun_input
    def cal_fun_input(self):
        data = pairdict[self] 
        result = dtw_parallel(data,self)
        return result
    
    done_x = imap_unordered_bar(cal_fun_input, pair, 6)
    return done_x

if __name__ == "__main__":
    dist_list = all_run_dtw(pairdict, pair)